## Micropython on a light bulb

This requires bitbanging as shown in https://github.com/xoseperez/my92xx
with teardown at https://tinkerman.cat/post/ailight-hackable-rgbw-light-bulb

Before building the Micropython, the following modules should be copied over:
```
cp micropython-mqtt/mqtt_as/mqtt_as.py micropython-fork/ports/esp8266/modules/
cp micropython-mqtt/mqtt_as/mqtt_as_timeout.py micropython-fork/ports/esp8266/modules/
cp -r micropython-lib/uasyncio/uasyncio/ micropython-fork/ports/esp8266/modules/
cp micropython-lib/uasyncio.core/uasyncio/core.py micropython-fork/ports/esp8266/modules/uasyncio/
cp micropython-lib/uasyncio.queues/uasyncio/queues.py micropython-fork/ports/esp8266/modules/uasyncio/
cp -r micropython-lib/umqtt.simple/umqtt/ micropython-fork/ports/esp8266/modules/
cp micropython-lib/umqtt.simple/umqtt/simple.py micropython-fork/ports/esp8266/modules/umqtt/
cp micropython-lib/umqtt.robust/umqtt/robust.py micropython-fork/ports/esp8266/modules/umqtt/
```

The code used for the function esp_my9291_write() is at the bottom, along with a backup of the arduino function also used.

The microcontroller is actually an ESP8285, so the flashing command is without the "-fm dio":

> esptool.py --port /dev/ttyUSB2 --baud 460800 write_flash --flash_size=detect 0 micropython/ports/esp8266/build-GENERIC/firmware-combined.bin




In [42]:
%serialconnect

Connecting to --port=/dev/ttyUSB2 --baud=115200 
Ready.


In [9]:
%sendtofile config.txt

wifiname     DoESLiverpool
wifipassword decafbad00
mqttbroker   10.0.100.1
pinDI        13
pinDCK       15
connection1  BV6000,bubblino,192.168.43.1


Sent 6 lines (149 bytes) to config.txt.


In [10]:
%sendtofile --source stdmqttas.py
%sendtofile --quiet --source utils.py

Sent 76 lines (2732 bytes) to stdmqttas.py.
Sent 32 lines (969 bytes) to utils.py.


In [45]:
%websocketconnect --password wpass ws://10.0.35.134:8266/

** WebSocket connected **
Password: 
WebREPL connected
>>> Ready.


In [46]:
%ls

Listing directory '/'.
      228    boot.py
      149    config.txt
     2992    main.py
     2732    stdmqttas.py
      969    utils.py
       15    webrepl_cfg.py


In [47]:
%sendtofile main.py

from mqtt_as import MQTTClient
from stdmqttas import fconfig, config, mqttconnecttask, callbackcmdtask
import uasyncio as asyncio
from stdmqttas import shortmac, topicstem, topicstatus
from machine import Pin
from esp import my9291_write
import time, itertools

pinDI = Pin(int(fconfig["pinDI"]), Pin.OUT) 
pinDCK = Pin(int(fconfig["pinDCK"]), Pin.OUT)
my9291_write(pinDI, pinDCK, b'\x00')  # 16 bits

topicrgbwhex = topicstem+"/rgbwhex"
topiccmd = (topicstem+"/cmd").encode()
topicreply = topicstem+"/reply"
topicbeat = topicstem+"/beat"

rgbw = bytearray(8)

def conv8hex(h):
    for i in range(8):
        rgbw[i] = int(h[i*2:i*2+2], 16)
        
def conv4hex(h):
    for i in range(4):
        rgbw[i*2] = int(h[i*2:i*2+2], 16)
        rgbw[i*2+1] = 0

def conv3hex(h):
    for i in range(3):
        rgbw[i*2] = int(h[i*2:i*2+2], 16)
        rgbw[i*2+1] = 0
    rgbw[6] = 0
    rgbw[7] = 0

def RGBWhex(rgbwhex):
    try:
        if len(rgbwhex) == 16:
            conv8hex(rgbwhex)
        elif len(rgbwhex) == 8:
            conv4hex(rgbwhex)
        elif len(rgbwhex) == 7 and rgbwhex[0] == ord("#"):
            conv3hex(rgbwhex[1:])
        elif len(rgbwhex) == 6:
            conv3hex(rgbwhex)
        else:
            return False
    except ValueError as e:
        return False
    except IndexError as e:
        return False
    print("my9291_write", rgbw)
    my9291_write(pinDI, pinDCK, rgbw)
    return True


def errflash():
    my9291_write(pinDI, pinDCK, b'\x09\x00\x00\x00\x00\x00\x00\x00')
    time.sleep_ms(200)
    my9291_write(pinDI, pinDCK, rgbw)
    
async def heartbeatofflineflashes(client):
    for j in itertools.count():
        while not client.isconnected():
            await asyncio.sleep_ms(1000)
            RGBWhex("00000500")
            await asyncio.sleep_ms(1000)
            RGBWhex("00000005")

        for i in range(8):
            RGBWhex("01000000" if (i%2) else "00100000")
            await asyncio.sleep_ms(100)
        
        for i in itertools.count():
            await asyncio.sleep_ms(1000)
            if not client.isconnected():
                break
            await client.publish(topicbeat, "%d-%d"%(j, i))

def callbackcmd(topic, msg, retained):
    print("callbackcmd", topic, msg)
    if topic == topiccmd:
        aloop.create_task(callbackcmdtask(client, topicreply, msg))
    else:
        if not RGBWhex(msg):
            print("error")
            errflash()

async def onconnecttask(client):
    ipnumber = client._sta_if.ifconfig()[0]
    await client.publish(topicstatus, ipnumber, retain=True)
    await client.subscribe(topiccmd)
    await client.subscribe(topicrgbwhex)
    print("subscribed to", topicrgbwhex)
            
config['subs_cb'] = callbackcmd
config['connect_coro'] = onconnecttask
client = MQTTClient(config)
client.DEBUG = True

print(topicstem)
aloop = asyncio.get_event_loop()
aloop.create_task(heartbeatofflineflashes(client))
aloop.create_task(mqttconnecttask(client))
aloop.run_forever()


Sent 105 lines (2991 bytes) to main.py.


In [3]:
%serialconnect

Connecting to --port=/dev/ttyUSB1 --baud=115200 
Ready.


In [ ]:

#define MY92XX_MODEL        MY92XX_MODEL_MY9291
#define MY92XX_CHIPS        2
#define MY92XX_DI_PIN       13
#define MY92XX_DCKI_PIN     15

#define MY92XX_RED          4
#define MY92XX_GREEN        3
#define MY92XX_BLUE         5

In [4]:
%sendtofile main.py

from machine import Pin
import time

DI = Pin(13, Pin.OUT)
DCK = Pin(15, Pin.OUT)
DI.value(0)
DCK.value(0)

def di_pulse(times):
    for i in range(1, times*2+1):
        DI.value(i&1)
def dck_pulse(times):
    for i in range(1, times*2+1):
        DCK.value(i&1)

def write8(data): 
    DCK.value(0)
    DI.value(data&0x80);
    DCK.value(1)
    DI.value(data&0x40);
    DCK.value(0)
    DI.value(data&0x20);
    DCK.value(1)
    DI.value(data&0x10);
    DCK.value(0)
    DI.value(data&0x08);
    DCK.value(1)
    DI.value(data&0x04);
    DCK.value(0)
    DI.value(data&0x02);
    DCK.value(1)
    DI.value(data&0x01);
    DCK.value(0)
    DI.value(0);

def set_cmd(command):
    time.sleep_us(12)

    # 12 pulses rising edge convert to command mode.
    di_pulse(12)
    time.sleep_us(12)

    write8(command)
    time.sleep_us(12)

    # Send 16 DI pulse，at 14 pulse's falling edge store CMD data, and
    # at 16 pulse's falling edge convert to duty mode.
    di_pulse(16);
    time.sleep_us(12)


channels = 4
dck_pulse(32)

# scatter|freq|bit_width|reaction|oneshot
cmd = (0x80&0x00) | (0x60&0x00) | (0x18&0x18) | (0x40&0x00) | (0x20&0x00)

set_cmd(cmd)

def send(r,g,b,w):
    time.sleep_us(12)
    write8(r)
    write8(g)
    write8(b)
    write8(w)
    time.sleep_us(12)
    di_pulse(8)
    time.sleep_us(12)

send(10,50,10,5)


Sent 70 lines (1337 bytes) to main.py.
